# 03. 배치 정규화 (Batch Normalization)
> 딥러닝 내부의 입력을 표준화 시켜주는 배치 정규화에 대해 알아봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 8]
- permalink: /batch_normalization
- exec: colab

이전 시간에 심층 신경망 학습에서는 DNN 학습에 있어서 적절한 활성화 함수 및 가중치 초기화 방법에 대해 알아보았습니다. 이번 포스팅에서는 그래디언트 소실(vanishing gradient)과 폭주(exploding) 문제를 해결하는 방법인 배치 정규화(BN, Batch Normalization)와 그래디언트 클리핑(Gradient Clipping), 그리고 학습 속도를 높일 수 있는 최적화(Optimization) 방법에 대해 알아봅시다.


<br> 

### 1. 배치 정규화 (BN, Batch Normalization)

#### 1.1. 배치정규화란?
다층 신경망 학습에서는 활성화 함수로는 ReLU를 사용하고 He 초기화를 통해 학습 초기 단계에서의 그래디언트 소실/폭주 문제를 줄일 수 있었지만, 이러한 문제가 학습하는 동안에 또 다시 발생할 가능성이 있습니다. <br><br>

2015년 Sergety Ioffe와 Christian Szegedy는 'Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift'라는 논문에서 배치 정규화(BN, Batch Normalization)를 제안했습니다.  배치 정규화는 각 층의 활성화 함수의 출력값 분포가 골고루 분포되도록 '강제'하는 방법으로, 각 층에서의 활성화 함수 출력값이 정규분포(normal distribution)를 이루도록 하는 방법입니다. 
<br><br>

즉, 학습하는 동안 이전 레이어에서의 가중치 매개변수가 변함에 따라 활성화 함수 출력값의 분포가 변화하는 내부 공변량 변화(Internal Covariate Shift) 문제를 줄이는 방법이 바로 배치 정규화 기법입니다. 배치 정규화는 아래의 그림과 같이 미니배치(mini-batch)의 데이터에서 각 feature(특성)별 평균(​mean)과 분산(​variance)을 구한 뒤 정규화(normalize) 해줍니다.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F99166C4B5BBDFFFA279D59)

일반적으로 배치 정규화는 아래의 그림과 같이 Dense Block이나 Convolutional Block 바로 다음, 그리고 활성화 함수를 통과하기 전에 배치 정규화(BN)레이어를 삽입하여 사용합니다.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile29.uf.tistory.com%2Fimage%2F994586445BBE000E15CC3D)

배치 정규화는 미니배치(mini-batch)를 단위로 데이터의 분포가 평균(​mean)이 0, 분산(variance)이 1이 되도록 정규화(normalization)합니다. 수식은 다음과 같습니다.

![](https://shuuki4.files.wordpress.com/2016/01/bn1.png)
<br><br>


#### 1.2. 배치정규화의 장점

Batch Normalization(BN)은 논문에서 실험했던 모든 딥 뉴럴넷의 성능이 크게 향상 시켰습니다. BN은 다음과 같은 장점들이 있습니다. 

- tanh나 sigmoid 같은 활성화 함수에 대해 그래디언트 소실(vanishing gradient)문제가 감소합니다.

- 가중치 초기화에 덜 민감합니다. 가중치 초기값에 크게 의존하지 않기 때문에 이전에 알아본 가중치 초기화 기법에 대해 크게 신경 쓰지 않아도 됩니다.

- 학습률(learning rate)를 크게 잡아도 gradient descent가 잘 수렴합니다.

- 오버피팅을 억제합니다. BN이 마치 Regularization 역할을 하기 때문에 드롭아웃(Dropout)과 같은 규제기법에 대한 필요성이 감소합니다.  하지만, BN로 인한 규제는 효과가 크지느ㄴ 않기 때문에 드롭아웃을 함께 사용하는 것이 좋습니다.
<br><br>

#### 1.3. Tensorflow에서 배치 정규화 사용하기

- MLP(다층퍼셉트론)에서 사용하기




In [10]:
# 데이터셋 로드

import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris


iris = load_iris()
feature = iris.data
label = iris.target
label = np.expand_dims(label, axis=1)

iris = np.concatenate([feature, label], axis=1)
np.random.shuffle(iris)

In [11]:
# 학습, 테스트 데이터셋 분할

feature = iris[:, :4]
label = iris[:, 4:]

split_point = int(0.8 * len(feature))
train_feature, train_label = feature[:split_point], label[:split_point]
test_feature, test_label = feature[split_point:], label[split_point:]

print(train_feature.shape, train_label.shape)
print(test_feature.shape, test_label.shape)

(120, 4) (120, 1)
(30, 4) (30, 1)


In [12]:
# 신경망 구현 (Class)

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, ReLU, BatchNormalization

class NeuralNetwork(Model):

    def __init__(self):
        super().__init__()
        self.relu = ReLU() # 액티베이션 함수 밖으로 빼기

        self.dense1 = Dense(512)
        self.bn1 = BatchNormalization()
        self.dense2 = Dense(512)
        self.bn2 = BatchNormalization()
        self.out = Dense(3, activation='softmax')

    def call(self, x):
      # 순서 : dense - bn - relu

        x = self.dense1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.dense2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.out(x)
        return x

In [14]:
# 컴파일 및 학습 진행

from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers


net = NeuralNetwork()
net.compile('adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

net.fit(train_feature, train_label, epochs=150)

Epoch 1/150
4/4 [==============================] - 0s 5ms/step - loss: 0.5184 - accuracy: 0.7500
Epoch 2/150
4/4 [==============================] - 0s 5ms/step - loss: 0.1501 - accuracy: 0.9500
Epoch 3/150
4/4 [==============================] - 0s 6ms/step - loss: 0.0692 - accuracy: 0.9750
Epoch 4/150
4/4 [==============================] - 0s 5ms/step - loss: 0.0776 - accuracy: 0.9667
Epoch 5/150
4/4 [==============================] - 0s 5ms/step - loss: 0.0470 - accuracy: 0.9917
Epoch 6/150
4/4 [==============================] - 0s 4ms/step - loss: 0.0459 - accuracy: 0.9833
Epoch 7/150
4/4 [==============================] - 0s 4ms/step - loss: 0.0341 - accuracy: 0.9917
Epoch 8/150
4/4 [==============================] - 0s 5ms/step - loss: 0.0535 - accuracy: 0.9667
Epoch 9/150
4/4 [==============================] - 0s 4ms/step - loss: 0.0639 - accuracy: 0.9667
Epoch 10/150
4/4 [==============================] - 0s 4ms/step - loss: 0.0581 - accuracy: 0.9750
Epoch 11/150
4/4 [===========

In [16]:
# 검증 과정

net.evaluate(test_feature, test_label)[1]

1/1 [==============================] - 0s 1ms/step - loss: 0.0747 - accuracy: 0.9333


0.9333333373069763

<br>

- CNN에서 사용하기

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, ReLU, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist

import numpy as np

In [7]:
class BatchNormCNN(Model):
    
    def __init__(self):
        super().__init__()
        self.relu = ReLU() # 액티베이션 함수 밖으로 빼기

        self.conv1 = Conv2D(32, kernel_size=(3, 3))
        self.bn1 = BatchNormalization()
        self.pool1 = MaxPooling2D(pool_size=2)

        self.conv2 = Conv2D(64, kernel_size=(3, 3))
        self.bn2 = BatchNormalization()
        self.pool2 = MaxPooling2D(pool_size=2)
        
        self.flatten = Flatten()
        self.hidden = Dense(256)
        self.out = Dense(10, activation='softmax')
        
        
    def call(self, x):
        # 순서 : conv- bn - relu
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)

        x = self.flatten(x)
        x = self.hidden(x)
        x = self.out(x)
        return x

In [3]:
(train_feature, train_label), (test_feature, test_label) = mnist.load_data()
train_feature = train_feature.astype(np.float32)
test_feature = test_feature.astype(np.float32)

print(train_feature.shape, train_label.shape)
print(test_feature.shape, test_label.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
train_feature = np.expand_dims(train_feature, axis=3)
test_feature = np.expand_dims(test_feature, axis=3)

In [5]:
print(train_feature.shape, train_label.shape)
print(test_feature.shape, test_label.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [6]:
train_feature /= 255.0
test_feature /= 255.0

In [8]:
cnn = BatchNormCNN()
cnn.build((None, *train_feature.shape[1:]))
cnn.summary()

Model: "batch_norm_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
batch_normalization (BatchNo multiple                  128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  18496     
_________________________________________________________________
batch_normalization_1 (Batch multiple                  256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple               

In [9]:
cnn.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

cnn.fit(train_feature, train_label, epochs=5, batch_size=32)

Epoch 1/5
1875/1875 [==============================] - 75s 40ms/step - loss: 0.1997 - accuracy: 0.9532
Epoch 2/5
1875/1875 [==============================] - 74s 39ms/step - loss: 0.0637 - accuracy: 0.9809
Epoch 3/5
1875/1875 [==============================] - 74s 39ms/step - loss: 0.0507 - accuracy: 0.9845
Epoch 4/5
1875/1875 [==============================] - 74s 39ms/step - loss: 0.0376 - accuracy: 0.9880
Epoch 5/5
1875/1875 [==============================] - 74s 40ms/step - loss: 0.0323 - accuracy: 0.9900
